In [5]:
import os 

In [1]:
class Hooklog:
# class SortedAPIList:
    path = None
    par = None
    rval = None
    
    digitname = None
    length = None
    hookli = None
    
    def __init__(self, path, par, rval):
        self.path = path
        self.par = par
        self.rval = rval
        self.hookli = list()
        self.length = 0
        
        self.__parseDigitname()
        self.__parseHooklog()
    
    def isExist(self):
        if os.path.isfile(self.path):
            return True
        else:
            return False
    
    def __len__(self):
        return self.length

    def __iter__(self):
        return iter(self.hookli)
    
    def __getitem__(self, key):
        return self.hookli[key]
    
    def __str__(self):
        return "class Hooklog, %s, par = %d, len = %d, digit name = %s" % (self.path, self.par, 
                                                                           self.length, self.digitname)
    
    # private functions
    def __parseDigitname(self):
        if self.path != "":
            self.digitname = self.path.split('/')[-1][:6] + '-' + self.path.split('_')[-1].split('.')[0]
    
    
    def __parseHooklog(self):
        if self.isExist():
            handle = open(self.path, 'rb')     # MIKE: change mode from 'r' to 'rb',
                                               # because windows cannot handel seek() in text(r) mode
            while 1:
                (tick, api) = self.__getNextPair(handle)
                if tick == 0: break # end of file or fail to read
                self.hookli.append((tick, api))
                self.length += 1               
            if handle: handle.close()
            #if logtype == 'H':
            self.hookli.sort(key = lambda tup: tup[0]) # sort by tick
        else:
            print self.path, ' is not exist'
            pass
        
    def __getNextPair(self, handle):
        tick = 0
        api = 'api'
        
        while 1:
            line = handle.readline()
            if not line: break # end of file
            if line[0] == '#': # start a new call 
                # and logtype == 'H' form 立垣
                tick = line[1:].strip()
                api =  handle.readline().strip()
                # MIKE: comment 20150811
                #if par == 1 and (api.startswith('Reg') or api == 'LoadLibrary' or api.startswith('=Reg')):
                #    api += handle.readline().strip()
                # MIKE: 20150811 new for Transformer
                # hack=> if api.startswith('=Reg'): api = api[1:]
                if self.par == 1 or self.rval == 1:
                    api = self._getParValue(api, handle)
                break
            else:
                continue
        
        return (tick, api)

    # protected functions
    # MIKE: 20160307, basic par. I move it out for better creating new class FeatureHooklog. 
    #                            It has to be protected!
    def _getParValue(self, api, handle):
        if api.startswith('Reg') or api == 'LoadLibrary':
            api += handle.readline().strip()
        return api
    
    # public functions
    def getWinSet(self, win):
        li_set = set()
        for i in range(self.length - win + 1): 
            key = ''
            for w in range(win):               
                key += self.li[i+w][1] 
            li_set.add(key)
        return li_set

In [2]:
class FeatureHooklog(Hooklog):
    
    isAll = None #是否考量全部 api 
    
    def __init__(self, path ,isAll):
        self.isAll = isAll
        Hooklog.__init__(self, , 1, 1)
        

    def __str__(self):
        return "class FeatureHooklog, %s, par = %d, len = %d, digit name = %s" % (
            self.path, self.par, self.length, self.digitname)
    
    
    # protected functions
    # MIKE: 20160307, in FeatureHooklog, new api is (api,firstpar,return)
    def _getParValue(self, api, handle):          
        parword = "#PR#"
        retword = "#PR#"
    
        if api not in API:
            print api, "not in API"
            return api
        
        while 1:
            pos = handle.tell()
            line = handle.readline()
            if not line: # reach to the end of file
                return api+parword+retword
            if line[0] == '#': # reach to next call
                handle.seek(pos)
                return api+parword+retword
        
            delimiter = line[1:].find('=') +1 # MIKE: for =Reg
            p = line[:delimiter]
            v = line[delimiter+1:].strip() #===value
        
            if not self.isAll:       
                if self.par == 1 and (p in APIPar1Ret[api][0]):
                    parword += self.__parTrans(api, p, v, 1) # v sometimes could be empty!
                if self.rval == 1 and (p in APIPar1Ret[api][1]):
                    retword += v
            else:
                if p in ['Return', 'result']:
                    retword += v
                else:
                    parword = parword + line + '?' 
                    
    # private functions
    def __parTrans(self, api, key, value, trans):
        t = api+key
    
        if not trans: return value #don't need trans
        
        FuncDict = self.__funcdict()
        
        if t in FuncDict:
            return FuncDict[t](value) # function dictionary
        else:
            print "no such transformation", api, key
            return value
        
    def __libTrans(self, value):
        DIR = LIB = EXT = "non"
        
        if value == "": return "NON\\NON\\NON"
        
        lvalue = value.lower()
        tokens = value.split('\\')
        
        # DIR
        if lvalue[1] != ':':
            DIR = "SYS"
        elif lvalue[0] == '\\':
            DIR = "LOC"
        else:
            for d in dir_dict:
                if d in lvalue:
                    DIR = dir_dict[d]
                    break

        # EXT
        t = tokens[-1].split('.')
        ext = t[-1]
        if ext == "" or len(t) == 1: # extension is . or no extension
            EXT = "DLL"
        else:
            EXT = ext.upper()
            
        # LIB
        LIB = tokens[-1].split('.')[0].lower()
    
        #return DIR+"@"+LIB+"@"+EXT+"("+value+")"
        return DIR+"@"+LIB+"@"+EXT

    def __fileTrans(self, value):
        DIR = EXT = ""
    
        # for empty value
        if value == "":return "NON@NON"
    
        lvalue = value.lower().replace('/', '\\')
        tokens = lvalue.split('\\')
    
        #DIR
        if lvalue.startswith("\\\\.\\"):
            DIR = "NET"
        elif lvalue[0] == '\\' or len(tokens) == 1:
            DIR = "LOC"
        else:
            for d in dir_dict:
                if d in lvalue:
                    DIR = dir_dict[d]
                    break
        if DIR == "": DIR = "ARB" # possbily c:
    
        # EXT
        t = tokens[-1].split('.')
        ext = t[-1]
        if ext == "" or len(t) == 1: # extension is . or no extension
            EXT = "NON"
        else:
            EXT = ext.upper()
    
        #return DIR+"@"+FIL+"@"+EXT+"("+value+")"
        return DIR+"@"+EXT
    
    def __keyTrans(self, value):
        HK = KEY = ""
        if value == "": return "NON@NON"
    
        lvalue = value.lower()
        tokens = lvalue.split('\\')
    
        # HK
        if tokens[0] in hkey_dict:
            HK = hkey_dict[tokens[0]]
        else:
            HK = "SUBK"
        
        # KEY
        subkey = lvalue[lvalue.find('\\'):]
        KEY = self.__remove_reg(subkey)
        KEY = self.__remove_fileName(KEY)
        KEY = self.__remove_mshist(KEY)
        KEY = self.__remove_sessioninfo(KEY)
    
        #return HK+"@"+KEY+"("+value+")"
        return HK+"@"+KEY
     
    def __thdTrans(self, value):
        return "NON"
    
    def __execTrans(self, value):
        return __fileTrans(value)

    def __cmdTrans(self, value):
        return value

    
    def __remove_reg(self, key):
        rvalue = ""
        tokens = key.split('\\')

        for t in tokens:
            if t.count('-') > 3:
                rvalue += "\\REGREG"
            else:
                rvalue += ('\\'+t)

        return rvalue

    def __remove_fileName(self, key):
        tok = key.split('\\')
        tok = [t for t in tok if t!='']
        for i,t in enumerate(tok):
            if '.' in t and len(t.split('.')) == 2:
                tok[i] = '.'+t.split('.')[1]
        return '\\'.join(tok)

    def __remove_mshist(self, key):
        tok = key.split('\\')
        tok = [t for t in tok if t!='']
        for i,t in enumerate(tok):
            if 'mshist' in t:
                tok[i] = 'mshistDate'
        return '\\'.join(tok)  

    def __remove_sessioninfo(self, key):
        tok = key.split('\\')
        parse_str = '\\'.join(tok)
        if 'software\\microsoft\\windows\\currentversion\\explorer\\sessioninfo' in parse_str:
            tok = parse_str.split('\\')
            if len(tok) == 7:
                tok[-1] = 'SESSIONID'
        return '\\'.join(tok)

    def __funcdict(self):
        FuncDict = {"LoadLibrarylpFileName": self.__libTrans,
                    "CreateProcesslpApplicationName": self.__execTrans,
                    "CreateProcessInternallpApplicationName": self.__execTrans,
                    "CreateThreaddwThreadId": self.__thdTrans, 
                    "WinExecCmdLine": self.__cmdTrans,
                    "CopyFilelpExistingFileName": self.__fileTrans,
                    "CreateFilehName": self.__fileTrans,
                    "DeleteFilefileName": self.__fileTrans,
                    "RegQueryValuehKey": self.__keyTrans,
                    "RegEnumValuehKey": self.__keyTrans,
                    "RegOpenKeyhKey": self.__keyTrans,
                    "RegCloseKeyhKey": self.__keyTrans,
                    "RegSetValuehKey": self.__keyTrans,
                    "RegCreateKeyhKey": self.__keyTrans,
                    "=RegDeleteKeyhKey": self.__keyTrans, #####
                    "RegDeleteKeyhKey": self.__keyTrans,
                    "RegDeleteValuehKey": self.__keyTrans
        }
        return FuncDict
    
    # public functions
    def getWinSet(self, win):
        li_set = set()
        for i in range(self.length - win + 1):
            key = list()
            for w in range(win):
                key.append(self.hookli[i+w][1]) # append (api, v, r)
            li_set.add(tuple(key))
        return li_set
    
    def getHkli_containTS(self):
        return self.hookli
    
    def getHkli_noContainTS(self):
        hookli_noTS = list()
        for i, (ts, api) in enumerate(self.hookli):
            hookli_noTS.append(api)
        
        return hookli_noTS
    
# Ordered!!
# dir_list = [
#     ("\\windows\\temp\\", "TMP"),  #==========
#     (":\\temp\\", "TMP"),  #==========
#     ("\\windows\\system32\\", "SYS"),
#     ("\\windows\\system\\", "SYS"),
#     ("\\windows\\", "WIN"), #==========
#     ("\\program files\\", "PRO"),
#     ("\\documents and settings\\", "USR"),
#     ("\\docume~1\\", "USR")
# ]
dir_dict = {
    "\\windows\\system32\\": "SYS",
    "\\windows\\system\\": "SYS",
    "\\program files\\": "PRO",
    #"\\documents and settings\\all users\\": "USR",
    "\\documents and settings\\": "USR",
    "\\docume~1\\": "USR"
}

hkey_dict = {
    "hkey_classes_root": "HKCR",
    "hkey_current_user": "HKCU",
    "hkey_local_machine": "HKLM",
    "hkey_users": "HKUS",
    "hkey_current_config": "HKCC"
}

APIPar1Ret = {
    "LoadLibrary": [["lpFileName"], ["Return","result"]],
    #"CreateProcess": [["lpApplicationName","lpCommandLine"], ["Return","result"]],
    "CreateProcess": [["lpApplicationName"], ["Return","result"]],
    #"CreateProcessInternal": [["lpApplicationName","lpCommandLine"], ["Return","result"]],
    "CreateProcessInternal": [["lpApplicationName"], ["Return","result"]],
    "OpenProcess": [[], ["Return","result"]], # it has procName, but might be empty
    "ExitProcess": [[], ["Return","result"]],
    "TerminateProcess": [[], ["Return","result"]],
    "WinExec": [["CmdLine"], ["Return","result"]],
    "CreateRemoteThread": [[], ["Return","result"]], # it has procName, but might be empty
    "OpenThread": [["dwThreadId"], ["Return","result"]],
    "CreateThread": [["dwThreadId"], ["Return","result"]],
    "TerminateThread": [["hThread"], ["Return","result"]],
    "CopyFile": [["lpExistingFileName"], ["Return","result"]], # include lpNewFileName?
    "CreateFile": [["hName"], ["Return","result"]], #MIKE: actually it is 'lpFileName' in spec.
    "WriteFile": [[], ["Return","result"]],
    "ReadFile": [[], ["Return","result"]],
    "DeleteFile": [["fileName"], ["Return","result"]], #MIKE: actually it is 'lpFileName' in spec.
    "CloseHandle": [[], ["Return","result"]],
    "RegOpenCurrentUser": [[], ["Return","result"]],
    "RegQueryValue": [["hKey"], ["Return","result"]],
    "RegEnumValue": [["hKey"], ["Return","result"]],
    "RegOpenKey": [["hKey"], ["Return","result"]],
    "RegCloseKey": [["hKey"], ["Return","result"]],
    "RegSetValue": [["hKey"], ["Return","result"]],
    "RegCreateKey": [["hKey"], ["Return","result"]],
    "=RegDeleteKey": [["hKey"], ["Return","result"]], # MIKE: misprint the '=' #============
    "RegDeleteKey": [["hKey"], ["Return","result"]], # MIKE: misprint the '='
    "RegDeleteValue": [["hKey"], ["Return","result"]],
    "WinHttpConnect": [["pswzServerName"], ["Return","result"]],
    "WinHttpCreateUrl": [["wszUrl"], ["Return","result"]],
    "WinHttpOpen": [["pwszUserAgent"], ["Return","result"]],
    "WinHttpOpenRequest": [["pwszObjectName"], ["Return","result"]],
    "WinHttpReadData": [["lpBuffer"], ["Return","result"]],
    "WinHttpSendRequest": [["pwszHeaders"], ["Return","result"]],
    "WinHttpWriteData": [["lpBuffer"], ["Return","result"]],
    "WinHttpGetProxyForUrl": [["lpcwszUrl"], ["Return","result"]],
    "InternetOpen": [["lpszAgent"], ["Return","result"]],
    "InternetConnect": [["lpszServerName"], ["Return","result"]],
    "HttpSendRequest": [["lpszHeaders"], ["Return","result"]],
    "GetUrlCacheEntryInfo": [["lpszUrlName"], ["Return","result"]],
}
API = APIPar1Ret.keys()

In [11]:
# path = 'data/family/725_.002 (7)/hooklogs/'
# hk = '221bdade3f1bd0dd211025fdc859055b_3248.trace.hooklog'
# f = path+hk

# fh = FeatureHooklog(f)
# fh.getHkli_noContainTS()
# fh.getHkli_containTS()
# # h = Hooklog(f,1,1)
# # h.getWinSet(1)
# # print fh1._getParValue('LoadLibrary',)

[('232460000', 'LoadLibrary#PR#SYS@imm32@DLL#PR#SUCCESS'),
 ('233010000', 'LoadLibrary#PR#SYS@lpk@DLL#PR#SUCCESS'),
 ('233320000', 'LoadLibrary#PR#SYS@gdi32@DLL#PR#SUCCESS'),
 ('233920000',
  'RegQueryValue#PR#HKLM@system\\currentcontrolset\\control\\session manager\\criticalsectiontimeout#PR#0'),
 ('233930000',
  'RegQueryValue#PR#HKLM@software\\microsoft\\ole\\rwlockresourcetimeout#PR#2'),
 ('234490000',
  'RegQueryValue#PR#HKCU@control panel\\desktop\\smoothscroll#PR#2'),
 ('234500000', 'LoadLibrary#PR#SYS@imm32@DLL#PR#SUCCESS')]